# Предсказание коэффициента восстановления золота из руды

В ходе данного проекта будут использоваться реальные данные, предоставленные компанией "Цифра". В результате работы будет построена модель, которая должна будет предсказать коэффициент восстановления золота из золотосодержащей руды. В нашем распоряжении данные с параметрами добычи и очистки золотосодержащей руды, уже разбитые на тренировочную и тестовую выборку, также имеется и полный набор данных, содержащий обе упомянутые выборки. В процессе работы будет произведена подготовка и анализ "сырых" данных и создание модели обучения. 

## Некоторая информация о данных

Данные индексируются датой и временем получения информации (признак date). Соседние по времени параметры часто похожи.<br>
Некоторые параметры недоступны, потому что замеряются и/или рассчитываются значительно позже. Из-за этого в тестовой выборке отсутствуют некоторые признаки, которые могут быть в обучающей. Также в тестовом наборе нет целевых признаков.<br>
Проведем **описание данных**:<br>
**Технологический процесс**<br>
Rougher feed — исходное сырье<br>
Rougher additions (или reagent additions) — флотационные реагенты: Xanthate, Sulphate, Depressant<br>
Xanthate — ксантогенат (промотер, или активатор флотации);<br>
Sulphate — сульфат (на данном производстве сульфид натрия);<br>
Depressant — депрессант (силикат натрия).<br>
Rougher process (англ. «грубый процесс») — флотация<br>
Rougher tails — отвальные хвосты<br>
Float banks — флотационная установка<br>
Cleaner process — очистка<br>
Rougher Au — черновой концентрат золота<br>
Final Au — финальный концентрат золота<br>
**Параметры этапов**<br>
air amount — объём воздуха<br>
fluid levels — уровень жидкости<br>
feed size — размер гранул сырья<br>
feed rate — скорость подачи<br>
**Расшифруем название признаков:**<br>
[этап].[тип_параметра].[название_параметра]<br>
Пример: rougher.input.feed_ag<br>
Возможные значения для блока [этап]:<br>
rougher — флотация<br>
primary_cleaner — первичная очистка<br>
secondary_cleaner — вторичная очистка<br>
final — финальные характеристики<br>
Возможные значения для блока [тип_параметра]:<br>
input — параметры сырья<br>
output — параметры продукта<br>
state — параметры, характеризующие текущее состояние этапа<br>
calculation — расчётные характеристики<br>

В качестве **метрики качества** будем использовать **sMAPE**. Нужно спрогнозировать сразу две величины:<br>
эффективность обогащения чернового концентрата rougher.output.recovery<br>
эффективность обогащения финального концентрата final.output.recovery<br>
Итоговая метрика складывается из двух величин:
$$sMAPE = 0.25 * sMAPE(rougher) + 0.75 * sMAPE(final)$$

## 1. Подготовка данных

### 1.1 Загрузка, обзор и предобработка данных

Сперва подключим необходимые в дальнейшей работе библиотеки:

In [ ]:
import pandas as pd
import numpy as np
try:
    import missingno as msno  
except:
    !pip install missingno
    import missingno as msno 
from sklearn.metrics import mean_absolute_error
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor

Теперь загрузим наши данные:

In [ ]:
try:
    gold_recovery_full = pd.read_csv('gold_recovery_full_new.csv')
except:
    gold_recovery_full = pd.read_csv('/datasets/gold_recovery_full_new.csv')
    
try:
    gold_recovery_train = pd.read_csv('gold_recovery_train_new.csv')
except:
    gold_recovery_train = pd.read_csv('/datasets/gold_recovery_train_new.csv')
    
try:
    gold_recovery_test = pd.read_csv('gold_recovery_test_new.csv')
except:
    gold_recovery_test = pd.read_csv('/datasets/gold_recovery_test_new.csv')

Проведем **первичный обзор** данных. Сперва настроим вывод - пусть при выводе отображается максимальное количество столбцов и строк:

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Теперь выведем первые 5 строк каждого датафрейма и его размеры.

In [ ]:
datasets = [gold_recovery_full, gold_recovery_train, gold_recovery_test]
datasets_string = ['gold_recovery_full', 'gold_recovery_train', 'gold_recovery_test']

for i in range(len(datasets)):
    print(f'Первые 5 строк набора {datasets_string[i]}:')
    display(datasets[i].head())
    display(datasets[i].shape)
    display()

Как можно убедиться, сумма строк обучающей и тестовой выборки действительно равняется количеству строк в исходном датафрейме. Кроме того, в тестовой выборке содержится на 34 столбца меньше, чем в предыдущих двух наборах. Чуть позже по ходу исследования узнаем, каких строк "лишилась" эта выборка.

Проверим данные на наличие дубликатов:

In [ ]:
for i in range(len(datasets)):
    print(f'Число дубликатов в {datasets_string[i]} - {datasets[i].duplicated().sum()}')
    print()

Видим, что наши наборы не содержат в себе дубликатов.

Теперь уточним, какой тип данных используется для записи информации о каждом столбце наших датафреймов:

In [ ]:
for i in range(len(datasets)):
    print(f'Тип данных для каждого столбца из датафрейма {datasets_string[i]}')
    print(datasets[i].dtypes)
    print()

Как видим, параметр 'date' имеет тип object. Для удобства переведем его в datetime:

In [ ]:
gold_recovery_full['date'] = pd.to_datetime(gold_recovery_full['date'], format='%Y-%m-%d %H:%M:%S')
gold_recovery_train['date'] = pd.to_datetime(gold_recovery_train['date'], format='%Y-%m-%d %H:%M:%S')
gold_recovery_test['date'] = pd.to_datetime(gold_recovery_test['date'], format='%Y-%m-%d %H:%M:%S')

Более того, будем использовать этот столбец как индекс для каждого датафрейма:

In [ ]:
gold_recovery_full = gold_recovery_full.set_index('date')
gold_recovery_train= gold_recovery_train.set_index('date')
gold_recovery_test = gold_recovery_test.set_index('date')

Теперь взглянем на ситуацию с пропусками в данных с помощью средств библиотеки missingno. Сперва оценим общее количество пропусков в исходных данных, затем - в обучающей и тестовой выборке по-отдельности:

In [ ]:
print('gold_recovery_full:')
msno.bar(gold_recovery_full, figsize = (15,30))

In [ ]:
print('gold_recovery_train:')
msno.bar(gold_recovery_train, figsize = (15,30))

In [ ]:
print('gold_recovery_test:')
msno.bar(gold_recovery_test, figsize = (15,20))

Похоже, что наши данные содержат сравнительно немного пропущенных значений. Кроме того, складывается впечатление, что в исходном и обучающем наборе данных пропусков больше, чем в тестовом.<br> 
Понять насколько данные подвержены пропускам очень важно для любого аналитического исследования, поэтому рассмотрим пропущенные значения подробнее. Напишем функцию, получающую на вход датафрейм и возвращающую Series, индексами которой являются 10 столбцов с наибольшим числом пропусков из этого датафрейма, а значениями - доля пропусков в этих столбцах: 

In [ ]:
def columns_w_most_na (df):
    missed_values = []
    for i in range(len(df.columns)):
        missed_values.append(df[df.columns[i]].isna().sum())
    missed_values_ser = pd.Series(data=missed_values, index=df.columns)
    missed_values_ser = missed_values_ser.sort_values(ascending=False).head(10)
    missed_values_ser = missed_values_ser / len(df)
    return missed_values_ser

Применим функцию к каждому из датафреймов:

In [ ]:
for i in range(len(datasets)):
    print(f'Доля пропусков в столбцах с наибольшим количеством пропущенных значений в наборе {datasets_string[i]}:')
    print(columns_w_most_na(datasets[i]))
    print()

Видим, что наше предположение оказалось адекватным данным - в изначальном и обучающем наборе пропусков в относительных значениях действительно больше, чем в тестовом наборе. Больше всего их в исходных данных в столбце 'secondary_cleaner.output.tail_sol' - почти 9%. Следующие за ним в топе столбцы имеют пропусков в разы меньше. В обучающем датафрейме тот же столбец лидирует по доле пропусков - 11% (доля выросла из-за самого принципа формирования обучающей выборки). По остальным столбцам наблюдается примерно та же динамика, что и в исходном наборе. Тестовая же выборка содержит очень мало пропусков - чуть меньше половины процента для наибольшего по пропускам столбца, дальше - еще меньше.

Посмотрим сколько столбцов в получившихся топах совпадают и какие это столбцы. Для этого напишем функцию, аргументами которой будут датафреймы. В процессе своей работы она будет с помощью написанной ранее функции получать топы столбцов по числу пропусков, а затем создавать список из тех столбцов, которые встречаются в обоих топах:

In [ ]:
def shared_columns_most_na (df_1, df_2): 
    top_na_columns_1 = columns_w_most_na(df_1)
    top_na_columns_2 = columns_w_most_na(df_2)
    shared_columns = [column for column in top_na_columns_1.index if column in top_na_columns_2.index]
    return shared_columns

Применим функцию к нашим наборам:

In [ ]:
for i in range(len(datasets)):    
    print(f'В {datasets_string[i]} и {datasets_string[i-1]} в топе по числу пропусков совпадают столбцы \
{shared_columns_most_na(datasets[i], datasets[i-1])}, всего их \
{len(shared_columns_most_na(datasets[i], datasets[i-1]))}.')
    print()

Как видим, топы столбцов по пропускам в исходных данных и обучающей выборке совпадают на 90%, что ожидаемо. Также, совпадают на 30% топы исходных данных (обучающей выборки) и тестовых данных.

Также проведем визуальную оценку местонахождения пропущенных значений в столбцах с самой высокой долей пропусков с помощью уже использовавшейся библиотеки missingno:

In [ ]:
print('gold_recovery_full:')
msno.matrix(gold_recovery_full[columns_w_most_na(gold_recovery_full).index])

In [ ]:
print('gold_recovery_train:')
msno.matrix(gold_recovery_train[columns_w_most_na(gold_recovery_train).index])

In [ ]:
print('gold_recovery_test:')
msno.matrix(gold_recovery_test[columns_w_most_na(gold_recovery_test).index])

Видим, что в исходной и тренировочной выборках в столбце 'secondary_cleaner.output.tail_sol' пропуски локализованы в нижней части датафрейма, то есть, находятся среди последних доступных нам по дате данных. Локализация пропусков наблюдается и в столбце 'rougher.state.floatbank10_e_air' - пробелы расположены в нижней трети данных. В районе середины периода наблюдений находятся пропуски столбца 'rougher.input.floatbank11_xanthate', более-менее равномерно распределены по данным пропуски в столбце 'primary_cleaner.output.concentrate_sol'. В столбце 'final.output.concentrate_sol' пропуски наблюдаются на начальном периоде.<br>
Также в тренировочной выборке видим примерно одну область концентрации пропусков для столбцов 'rougher.input.feed_pb' и 'final.output.tail_pb' (для последнего признака область пропусков совпадает и с исходным набором).<br>
В тестовой выборке пропуски выглядят менее скученно, кажется, будто их распределение по столбцам более случайно.<br>
Вероятно, стоит уточнить у заказчика, по каким причинам ситуация с пропусками может выглядеть именно так, в исходной и обучающей выборках их локализация в некоторых столбцах выглядит неслучайно. 

От визуальной оценки пропусков перейдем к их обработке. Для имеющихся обучающих и тестовых выборок представляются удачными два способа обработки пропусков - удаление и заполнение вдоль строк. Второй способ имеет смысл, так как из ТЗ мы знаем, что "Соседние по времени параметры часто похожи", поэтому мы можем использовать предыдущее знеачение параметра для заполнения следующего за ним пропуска. Выберем второй способ, так как он представляется более подходящим нашим данным:

In [ ]:
gold_recovery_train = gold_recovery_train.fillna(method='ffill')
gold_recovery_test = gold_recovery_test.fillna(method='ffill')

### 1.2 Проверка правильности рассчета эффективности обогащения

Удостоверимся, что эффективность обогащения рассчитана верно. Заново вычислим её на обучающей выборке для признака rougher.output.recovery, после чего рассчитаем MAE между вашими расчётами и значением признака.

Начнем с написания функции, рассчитывающей эффективность обогащения:

In [ ]:
def recovery (c, f, t):
    return ((c * (f - t)) / (f * (c - t))) * 100

Заведем необходимые для рассчетов переменные:

In [ ]:
recovery_train_target = gold_recovery_train['rougher.output.recovery']
с_train = gold_recovery_train['rougher.output.concentrate_au']
f_train = gold_recovery_train['rougher.input.feed_au']
t_train = gold_recovery_train['rougher.output.tail_au']

Рассчитаем эффективность обогащения:

In [ ]:
recovery_train_calculated = recovery(с_train, f_train, t_train)

Теперь рассчитаем метрику MAE:

In [ ]:
mae_recovery_train = mean_absolute_error(recovery_train_target, recovery_train_calculated)
print(f'Метрика MAE равна {mae_recovery_train}')

Видим, что MAE находится в нуле, что значит, что эффективность обогащения для признака rougher.output.recovery рассчитана правильно. 

### 1.3 Анализ признаков, недоступных в тестовой выборке

Посмотрим, какие столбцы, имеющиеся в исходных данных, отсутствуют в тестовой выборке:

In [ ]:
missed_test_columns = [column for column in gold_recovery_train.columns if column not in gold_recovery_test.columns]
print(missed_test_columns)

Эти столбцы-признаки, во-первых, содержат в себе параметры получившегося продукта для этапов флотации, первичной и вторичной очистки, финального продукта. Сами параметры включают в себя долю серебра (ag), свинца (pb), sol (кажется, что это коллоидный раствор), золота (au) после каждого этапа; долю тех же элементов в отвальных хвостах; эффективность обогащения. Во-вторых, они содержат в себе расчетные характеристики для этапа флотации, а именно - концентрацию сульфата (sulfate, сульфида натрия) по отношению к золоту, отношение сульфата к золоту во флотационных установках 10 и 11 для сырья, отношение золота к свинцу. 

## 2. Анализ данных

### 2.1 Анализ концентрации металлов на различных этапах отчистки

Проанализируем основные статистики, касающиеся концентрации каждого металла на этапах очистки, сделаем некоторые заключения о распределении соответствующих значений.

Сперва создадим список с названиями признаков на каждом из этапов отчистки (rougher, primary_cleaner, secondary_cleaner, final), а затем на его основе создадим списки с названиями признаков для каждого металла по-отдельности:

In [ ]:
metal_concentrate_cols = ['rougher.input.feed_au', 'rougher.input.feed_ag', 'rougher.input.feed_pb', 
                    'rougher.output.concentrate_au', 'rougher.output.concentrate_ag', 'rougher.output.concentrate_pb', 
                    'primary_cleaner.output.concentrate_au', 'primary_cleaner.output.concentrate_ag', 
                    'primary_cleaner.output.concentrate_pb', 'final.output.concentrate_au', 'final.output.concentrate_ag', 
                    'final.output.concentrate_pb']

au_concentrate_cols = [col for col in metal_concentrate_cols if 'au' in col]
ag_concentrate_cols = [col for col in metal_concentrate_cols if 'ag' in col]
pb_concentrate_cols = [col for col in metal_concentrate_cols if 'pb' in col]

Теперь, с помощью метода describe() просмотрим статистики по каждому из металлов на разных этапах обработки, а с помощью средств библиотеки plotly создадим ящики с усами. Выставим для них ограничения по оси y, чтобы лучше видеть основную часть графика. Чтобы увидеть выбросы, можно воспользоваться меню в правом верхнем углу графика (инструменты Zoom in, Zoom out) и возможностью курсором выделять область на графике, которую хочется рассмотреть детальнее.

In [ ]:
gold_recovery_train[au_concentrate_cols].describe()

In [ ]:
fig = px.box(gold_recovery_train[au_concentrate_cols], y=au_concentrate_cols, 
            title='Распределение концентрации золота на каждом этапе очистки')
fig.update_yaxes(range=[2,51.5])
fig.show()

Как и должно быть, с каждым новым этапом среднее содержание золота повышается. Распределение значений нормальное с очень небольшой скошенностью влево на последних трех этапах. Данные достаточно плотно сконцентрированны вокруг среднего, о чем говорит относительно небольшое значение стандартного отклонения и небольшой размер интервала между нижним и верхним квартилем. 

In [ ]:
gold_recovery_train[ag_concentrate_cols].describe()

In [ ]:
fig = px.box(gold_recovery_train[ag_concentrate_cols], y=ag_concentrate_cols, 
            title='Распределение концентрации серебра на каждом этапе очистки')
fig.update_yaxes(range=[1,19])
fig.show()

С серебром ситуация несколько иная. Мы видим увеличение его концентрации после этапа флотации, а затем происходит ее постепенное уменьшение на следующих этапах очистки. Схожая динамика наблюдается и на максимальных значениях концентрации на каждом из этапов. Наблюдается небольшая общая скошенность данных вправо. 

In [ ]:
gold_recovery_train[pb_concentrate_cols].describe()

In [ ]:
fig = px.box(gold_recovery_train[pb_concentrate_cols], y=pb_concentrate_cols, 
            title='Распределение концентрации свинца на каждом этапе очистки')
fig.update_yaxes(range=[0,16.5])
fig.show()

Динамика изменения значений среднего для концентрации свинца схожа направлением с той же динамикой для золота, однако скорость роста явно меньше. Иными словами, концентрация свинца на каждом этапе несильно растет. Последние два этапа характеризуются почти одинаковым значением медианы при сужающемся на финальном этапе межквартильном размахе - данные сильнее "скучковались" вокруг средних значений. Разброс данных так же не очень велик, выраженной скошенности не наблюдается. 

### 2.2 Сравнение распределений размеров гранул сырья на обучающей и тестовой выборках

Информация о размере гранул сырья находится в столбце 'rougher.input.feed_size'. Сохраним информацию о значениях этого признака в обучающей и тестовой выборках в соответствующих переменных:

In [ ]:
feed_size_train = gold_recovery_train['rougher.input.feed_size']
feed_size_test = gold_recovery_test['rougher.input.feed_size']

Теперь, средствами библиотеки plotly построим для этих данных ящики с усами, дополнительно отобразив на них среднее значение (пунктир):

In [ ]:
fig = make_subplots(rows=1, cols=2)

fig.append_trace(go.Box(
   y=feed_size_train.tolist(),
    name='Обучающая выборка',
    boxmean=True,
), row=1, col=1)

fig.append_trace(go.Box(
   y=feed_size_test.tolist(),
    name='Тестовая выборка',
    boxmean=True,
), row=1, col=2)

fig.update_yaxes(range=[20,95])
fig.update_layout(title_text='Распределения размеров гранул сырья на обучающей и тестовой выборке')
fig.show()

Видим, что обра набора данных распределены похожим образом. Медианные значения обучающей и тестовой выборки - 55 и 50 соответственно, средние значения 60 и 55. Таким образом, медиана и среднее обоих распределений не выходят за пределы межквартильного размаха друг друга. Налицо также положительная асимметрия в данных (среднее в обоих случаях превосходит медиану). Интервал, задаваемый нижним и верхним усами у обоих графиков почти идентичен - [24,92] для обучающей выборки, [25,90] для тестовой. Интервал же межквартильного размаха для обучающей и тестовой выборок - [49,66] и [44,62]. Если рассмотреть размах выбросов, то для обучающей выборки он несколько сдвинут вверх - [10,485] против [0,392] у тестовой выборки. Большая часть выбросов обоих распределений, таким образом, располагается выше верхнего уса. Итак, проанализировав графики, скажем, что **распределения отличаются друг от друга не сильно**.

### 2.3 Исследование суммарной концентрации всех веществ на разных стадиях

Рассмотрим общую концентрацию всех веществ в сырье, в черновом и финальном концентратах (по ТЗ).

Для начала соберем список необходимых признаков. В качестве веществ рассмотрим золото (au), серебро (ag), свинец (pb) и sol. Необходимые нам этапы и типы параметров - rougher.input (сырье), rougher.output (черновой концентрат) и final.output (финальный концентрат). Соберем соответствующие названия столбцов в общий список и в списки, соответствующие каждому этапу:

In [ ]:
matter_concentrate_cols = ['rougher.input.feed_au', 'rougher.input.feed_ag', 'rougher.input.feed_pb', 
                           'rougher.input.feed_sol', 'rougher.output.concentrate_au', 'rougher.output.concentrate_ag', 
                           'rougher.output.concentrate_pb', 'rougher.output.concentrate_sol', 
                           'final.output.concentrate_au',  'final.output.concentrate_ag', 
                           'final.output.concentrate_pb', 'final.output.concentrate_sol']

rougher_input_cols = ['rougher.input.feed_au', 'rougher.input.feed_ag', 'rougher.input.feed_pb', 'rougher.input.feed_sol']
rougher_output_cols = ['rougher.output.concentrate_au', 'rougher.output.concentrate_ag', 'rougher.output.concentrate_pb', 
                       'rougher.output.concentrate_sol']
final_output_cols = ['final.output.concentrate_au',  'final.output.concentrate_ag', 'final.output.concentrate_pb', 
                     'final.output.concentrate_sol']

Теперь создадим новый датафрейм, полученный из обучающей выборки по названиям нужных столбцов, просуммируем значения столбцов для каждого этапа и полученное значение запишем в новый столбец, после удалив столбцы-слагаемые:

In [ ]:
matter_concentrate = gold_recovery_train[matter_concentrate_cols].copy()

matter_concentrate['rougher.input.sum'] = matter_concentrate[rougher_input_cols].sum(axis=1)
#matter_concentrate = matter_concentrate.drop(rougher_input_cols, axis=1)

matter_concentrate['rougher.output.sum'] = matter_concentrate[rougher_output_cols].sum(axis=1)
#matter_concentrate = matter_concentrate.drop(rougher_output_cols, axis=1)

matter_concentrate['final.output.sum'] = matter_concentrate[final_output_cols].sum(axis=1)
#matter_concentrate = matter_concentrate.drop(final_output_cols, axis=1)

Теперь получим статистики по каждому из столбцом-сумм получившегося датафрейма и визуализируем распределение с помощью ящиков с усами:

In [ ]:
matter_concentrate[['rougher.input.sum', 'rougher.output.sum', 'final.output.sum']].describe()

In [ ]:
fig = make_subplots(rows=1, cols=3)

fig.append_trace(go.Box(
   y=matter_concentrate['rougher.input.sum'].tolist(),
   name='Вещества в сырье',
   boxmean=True,
), row=1, col=1)

fig.append_trace(go.Box(
   y=matter_concentrate['rougher.output.sum'].tolist(),
   name='Вещества в черн. концентрате',
   boxmean=True, 
), row=1, col=2)

fig.append_trace(go.Box(
   y=matter_concentrate['final.output.sum'].tolist(),
   name='Вещества в фин. концентрате',
   boxmean=True, 
), row=1, col=3)

fig.update_yaxes(range=[35,90])
fig.update_layout(title_text='Распределения суммарной концентрации веществ на разных стадиях')
fig.show()

Как видно, медианная суммарная концентрация веществ вырастает от сырья к черновому концентрату, оставаясь примерно на том же уровне в финальном концентрате. Рост медианы и среднего от этапа сырья к этапу чернового концентрата вызваны процессом обогащения. Данные несколько скошены влево (особенно в случае с распределением для чернового концентрата) с большим числом выбросов снизу от конца усов. Величина межквартильного размаха значений для сырья и чернового концентрата примерно совпадают (в районе 9), явным образом сужаясь в финальном концентрате (в районе 3). Межквартильный размах на этом этапе вкладывается в межквартильный размах в предыдущем - видно, как после очисток чернового концентрата медиана и среднее остались примерно теми же, а размах сократился почти троекратно. Скорее всего, сужение это происходит засчет уменьшения концентрации всех веществ, кроме золота, что является основной целью всего процесса флотации и очистки. 

Для более детальной оценки ситуации с аномалиями дополнительно посмотрим гистограммы для соответствующих признаков, кроме того, дополнительно визуализируем распределение горизонтальными ящиками с усами над графиком:

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.histogram(matter_concentrate, x=['rougher.input.sum', 'rougher.output.sum', 'final.output.sum'],
                  title='Гистограммы распределений суммарной концентрации веществ на разных стадиях', 
                  marginal='box')
fig.show()

Как видим, в нуле образовался столбик аномалий для всех трех признаков со значениями 0 и 0.04. Удалим соответствующие значения из нашего обучающего набора, для чего сначала соберем их в новый датафрейм:

In [ ]:
matter_concentrate_outliers = matter_concentrate[(matter_concentrate['rougher.input.sum'] <= 0.04) | 
                                                 (matter_concentrate['rougher.output.sum'] <= 0.04) | 
                                                 (matter_concentrate['final.output.sum'] <= 0.04)]

И затем удалим строки в обучающей выборке, совпадающие по индексам со строками в "аномальном" датафрейме":

In [ ]:
gold_recovery_train = gold_recovery_train.drop(gold_recovery_train.loc[matter_concentrate_outliers.index].index, axis=0)

## 3. Построение модели

### 3.1 Функция для рассчета sMAPE

Напишем функцию, с помощью которой сможем посчитать симметричное среднее абсолютное процентное отклонение (sMAPE) для двух объектов pandas Series. Поделим модуль разности этих объектов на среднее суммы их модулей, затем вычислим среднее получившегося объекта и умножим на 100, чтобы получить результат в процентах: 

In [ ]:
def smape (target, predicted):
    return (abs(target - predicted) / ((abs(target) + abs(predicted)) / 2)).mean() * 100

### 3.2 Обучение моделей

На финальном этапе работы нам необходимо предсказать параметры rougher.output.recovery и final.output.recovery по данным, которыми мы располагаем до начала этапа флотации, то есть, по параметрам сырья (rougher.input.). При этом, обучать модель на тренировочных данных мы можем только по тем параметрам, которые есть в тестовой выборке. Принимая во внимание эти ограничения, создадим список подходящих для обучения моделей параметров:

In [ ]:
feature_cols = ['rougher.input.feed_ag', 'rougher.input.feed_pb', 'rougher.input.feed_rate',
       'rougher.input.feed_size', 'rougher.input.feed_sol', 'rougher.input.feed_au', 'rougher.input.floatbank10_sulfate',
       'rougher.input.floatbank10_xanthate','rougher.input.floatbank11_sulfate', 'rougher.input.floatbank11_xanthate']

Теперь добавим в тестовую выборку целевые признаки:

In [ ]:
gold_recovery_test['rougher.output.recovery'] = gold_recovery_full.loc[gold_recovery_test.index, 'rougher.output.recovery']
gold_recovery_test['final.output.recovery'] = gold_recovery_full.loc[gold_recovery_test.index, 'final.output.recovery']

Разделим наши выборки:

In [ ]:
features_train = gold_recovery_train[feature_cols]
target_train_rougher = gold_recovery_train['rougher.output.recovery']
target_train_final = gold_recovery_train['final.output.recovery']

features_test = gold_recovery_test[feature_cols]
target_test_rougher = gold_recovery_test['rougher.output.recovery']
target_test_final = gold_recovery_test['final.output.recovery']

Начнем обучение с **линейной регрессии**. Чтобы сразу оценивать качество модели с помощью метрики sMAPE воспользуемся инструментом make_scorer библиотеку sklearn. Не забудем, что greater_is_better=False умножает метрику на -1, поэтому оценивать результаты валидации будем по модулю:

In [ ]:
smape_scorer = make_scorer(smape, greater_is_better=False)

In [ ]:
model = LinearRegression()
print(abs(cross_val_score(model, features_train, target_train_rougher, scoring=smape_scorer, cv=5).mean()))
print(abs(cross_val_score(model, features_train, target_train_final, scoring=smape_scorer, cv=5).mean()))

Средняя метрика после кросс-валидации для модели линейной регрессии, предсказывающей целевой признак эффективности обогащения чернового концентрата, равна 6.55, для эффективности обогащения финального концентрата - 9.49. 

Теперь подберем параметры и посчитаем метрики для модели **решающего дерева**:

In [ ]:
model = DecisionTreeRegressor()
parameters = {'random_state' : [12345],
              'max_depth': range(1, 11)}
grid = GridSearchCV(model, parameters, scoring=smape_scorer, cv=5)

Выведем лучшие гиперпараметры после гридсерча и среднюю метрику sMAPE по итогам кросс-валидации для двух целевых параметров:

In [ ]:
grid.fit(features_train, target_train_rougher)
print(grid.best_params_)
print(abs(grid.best_score_))

In [ ]:
grid.fit(features_train, target_train_final)
print(grid.best_params_)
print(abs(grid.best_score_))

Итак, наилучший гиперпараметр max_depth в случае предсказания для чернового концентрата - 4, среднее значение sMAPE - 6.45. В случае с предсказанием для финального концентрата значение глубины - 1, среднее sMAPE - 9.61.

Ту же процедуру проведем для модели **случайного леса**:

In [ ]:
model = RandomForestRegressor()
parameters = {'random_state' : [12345],
              'n_estimators': range(10, 51, 10),
              'max_depth': range(1, 13, 2),}
grid = GridSearchCV(model, parameters, scoring=smape_scorer, cv=5)

In [ ]:
grid.fit(features_train, target_train_rougher)
print(grid.best_params_)
print(abs(grid.best_score_))

In [ ]:
grid.fit(features_train, target_train_final)
print(grid.best_params_)
print(abs(grid.best_score_))

Как видим, наилучшие гиперпараметры для предсказания чернового концентрата: n_estimators - 10, max_depth - 9. Среднее значений sMAPE после кросс-валидации - 6.37. А для предсказания финального концентрата: n_estimators - 20, max_depth - 5, среднее sMAPE - 9.31.

По итогу обучения можем сказать, что наилучшая модель для предсказания эффективности обогащения чернового и финального концентратов - случайный лес для обоих целевых признаков.

Теперь проведем проверку лучших моделей на тестовой выборке. Начнем с предсказания чернового концентрата:

In [ ]:
model = RandomForestRegressor(n_estimators=10, max_depth=9, random_state=12345)
model.fit(features_train, target_train_rougher)
predictions_rougher = model.predict(features_test)
smape_rougher = smape(target_test_rougher, predictions_rougher)

model = RandomForestRegressor(n_estimators=20, max_depth=5, random_state=12345)
model.fit(features_train, target_train_final)
predictions_final = model.predict(features_test)
smape_final = smape(target_test_final, predictions_final)

Выведем значение метрик sMAPE для черного и финального концентратов тестовой выборки:

In [ ]:
print(smape_rougher, smape_final)

Как видим, на тестовой выборке удалось достичь следующих значений: 8.22 для чернового концентрата и 9.29 для финального.

Посчитаем итоговое симметричное среднее абсолютное процентное отклонение:

In [ ]:
0.25 * smape_rougher + 0.75 * smape_final

Итак, итоговое значение метрики sMAPE - 9.02, проведем проверку модели на адекватность с помощью sklearn.dummy.DummyRegressor:

In [ ]:
dummy_model = DummyRegressor(strategy="mean")
dummy_model.fit(features_train, target_train_rougher)
dummy_predictions_rougher = dummy_model.predict(features_test)
dummy_smape_rougher = smape(target_test_rougher, dummy_predictions_rougher)

dummy_model = DummyRegressor(strategy="mean")
dummy_model.fit(features_train, target_train_final)
dummy_predictions_final = dummy_model.predict(features_test)
dummy_smape_final = smape(target_test_final, dummy_predictions_final)

Выведем метрику sMAPE, оценивающую предсказания DummyRegressor для чернового и финального концентрата:

In [ ]:
print(dummy_smape_rougher, dummy_smape_final)

Значения метрики несколько ухудшились, 9.07 для чернового концентрата и 10.08 для финального. Посчитаем итоговую sMAPE:

In [ ]:
0.25 * dummy_smape_rougher + 0.75 * dummy_smape_final

Итоговая метрика составила 9.83, что несколько выше метрики модели случайного леса для регрессии, которая составляла 9.02. Вероятно, есть другие модели, более подходящие для этой задачи, которые дадут более ощутимую разницу в оценке предсказательной силы. В любом случае, наша модель случайного леса все еще справляется со своей задачей несколько лучше, чем дамми-модель регрессии. 

В рамках данного исследования мы готовили прототип модели машинного обучения для компании «Цифра». <br>
На первом этапе мы ознакомились с имеющимися данными по процессу получения золота из руды, оценили типы данных, число пропусков, наличие дубликатов. Провели соответствующую обработку. Затем мы оценили расчет эффективности обогащения для чернового сырья и проанализировали состав тестовой выборки.<br>
Второй этап состоял из анализа некоторых особенностей наших данных. Сперва мы оценили изменение концентрации каждого металла по ходу очистки, затем сравнили распределения размеров гранул сырья на обучающей и тестовой выборках, что было важно сделать для того, чтобы убедиться в адекватности дальнейшей работы моделей машинного обучения. Наконец, мы исследовали суммарную концентрацию всех веществ в сырье, черновом и финальном концентратах.<br>
Третий этап включал в себя работу с моделями. Начали мы с написания функции подсчета метрики sMAPE, которую в дальнейшем использовали для оценки предсказаний модели. Затем, пользуясь гридсерчем и кроссвалидацией, мы провели подбор гиперпараметров и обучение моделей линейной регрессии, решающего дерева и случайного леса. В финале исследования мы осуществили предсказание целевых признаков на тестовой выборке и рассчитали итоговое значение sMAPE. 